In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow.keras.layers as Layers
import tensorflow_hub as hub
import os

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
files = [
    os.path.join(
      os.path.join(os.getcwd(), 'drive/MyDrive/sports/train/baseball'),
      f
    )
    for f in os.listdir(os.path.join(os.getcwd(), 'drive/MyDrive/sports/train/baseball'))
]

In [ ]:
print(len(tf.config.list_physical_devices('GPU')))

1


In [ ]:
generator = ImageDataGenerator(rescale=1./255, dtype='float32')

train = generator.flow_from_directory(
    '/content/drive/MyDrive/sports/train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=True
)

augmentation = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal'),
    tf.keras.layers.experimental.preprocessing.RandomRotation(0.1),
    tf.keras.layers.experimental.preprocessing.RandomZoom(0.1)
  ], name='augmentation'
)

val = generator.flow_from_directory(
    '/content/drive/MyDrive/sports/valid',
    target_size=(224, 224),
    batch_size=16,
    class_mode='categorical',
    shuffle=False
)

Found 13492 images belonging to 100 classes.
Found 500 images belonging to 100 classes.


In [ ]:
earlystop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.0001, patience = 5, verbose=1, mode='auto', restore_best_weights=True)
lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, verbose=1, mode='auto')

model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(224,224,3)),
    augmentation,
    hub.KerasLayer("https://www.kaggle.com/models/google/efficientnet-v2/frameworks/TensorFlow2/variations/imagenet1k-b0-classification/versions/2", trainable=False),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(len(train.class_indices), activation=tf.keras.activations.softmax)
])

model.compile(
    loss=tf.keras.losses.categorical_crossentropy,
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
    metrics=["Accuracy"])

with tf.device(tf.test.gpu_device_name()):
  model.fit(train,validation_data=val,epochs=5,callbacks=[earlystop,lr], steps_per_epoch=len(train), validation_steps=int(len(val)*0.1))

Epoch 1/5
422/422 [==============================] - 3344s 8s/step - loss: 2.0130 - Accuracy: 0.5935 - val_loss: 0.3499 - val_Accuracy: 0.8958 - lr: 0.0100
Epoch 2/5
422/422 [==============================] - 49s 116ms/step - loss: 0.9055 - Accuracy: 0.7481 - val_loss: 0.3057 - val_Accuracy: 0.8750 - lr: 0.0100
Epoch 3/5
422/422 [==============================] - 50s 118ms/step - loss: 0.8303 - Accuracy: 0.7702 - val_loss: 0.3849 - val_Accuracy: 0.8750 - lr: 0.0100
Epoch 4/5
422/422 [==============================] - 51s 121ms/step - loss: 0.8029 - Accuracy: 0.7738 - val_loss: 0.2662 - val_Accuracy: 0.9167 - lr: 0.0100
Epoch 5/5
422/422 [==============================] - 51s 121ms/step - loss: 0.7604 - Accuracy: 0.7868 - val_loss: 0.2660 - val_Accuracy: 0.8958 - lr: 0.0100


In [ ]:
earlystop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.0001, patience = 5, verbose=1, mode='auto', restore_best_weights=True)
lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, verbose=1, mode='auto')

with tf.device(tf.test.gpu_device_name()):
  model.fit(train,validation_data=val,epochs=20,callbacks=[earlystop,lr], steps_per_epoch=len(train), validation_steps=int(len(val)*0.1))

Epoch 1/20
422/422 [==============================] - 64s 152ms/step - loss: 0.2608 - Accuracy: 0.9205 - val_loss: 0.1351 - val_Accuracy: 0.9375 - lr: 4.0000e-04
Epoch 2/20
422/422 [==============================] - 63s 150ms/step - loss: 0.2633 - Accuracy: 0.9202 - val_loss: 0.1309 - val_Accuracy: 0.9375 - lr: 4.0000e-04
Epoch 3/20
422/422 [==============================] - 61s 145ms/step - loss: 0.2495 - Accuracy: 0.9238 - val_loss: 0.1167 - val_Accuracy: 0.9583 - lr: 4.0000e-04
Epoch 4/20
422/422 [==============================] - 65s 154ms/step - loss: 0.2503 - Accuracy: 0.9248 - val_loss: 0.1457 - val_Accuracy: 0.9167 - lr: 4.0000e-04
Epoch 5/20
422/422 [==============================] - ETA: 0s - loss: 0.2446 - Accuracy: 0.9229
Epoch 5: ReduceLROnPlateau reducing learning rate to 7.999999215826393e-05.
422/422 [==============================] - 62s 146ms/step - loss: 0.2446 - Accuracy: 0.9229 - val_loss: 0.1433 - val_Accuracy: 0.9375 - lr: 4.0000e-04
Epoch 6/20
422/422 [=========

In [ ]:
model.evaluate(test)

500/500 [==============================] - 38s 74ms/step - loss: 0.2743 - Accuracy: 0.9180


[0.27427488565444946, 0.9179999828338623]

In [ ]:
url = "https://www.kaggle.com/models/tensorflow/resnet-50/frameworks/TensorFlow2/variations/classification/versions/1"
earlystop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.0001, patience = 5, verbose=1, mode='auto', restore_best_weights=True)
lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, verbose=1, mode='auto')

model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(224,224,3)),
    augmentation,
    hub.KerasLayer(url, trainable=False, input_shape=(224,224,3)),
    tf.keras.layers.Dense(1000, activation=tf.keras.activations.relu),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(len(train.class_indices), activation=tf.keras.activations.softmax)
])

model.compile(
    loss=tf.keras.losses.categorical_crossentropy,
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
    metrics=["Accuracy"])

with tf.device(tf.test.gpu_device_name()):
  model.fit(train,validation_data=val,epochs=10,callbacks=[earlystop,lr], steps_per_epoch=len(train), validation_steps=int(len(val)*0.1))

Epoch 1/10
422/422 [==============================] - 84s 177ms/step - loss: 2.3808 - Accuracy: 0.3806 - val_loss: 1.9508 - val_Accuracy: 0.5625 - lr: 0.0100
Epoch 2/10
422/422 [==============================] - 64s 152ms/step - loss: 1.9186 - Accuracy: 0.4804 - val_loss: 1.8154 - val_Accuracy: 0.5833 - lr: 0.0100
Epoch 3/10
422/422 [==============================] - 66s 155ms/step - loss: 1.8080 - Accuracy: 0.5052 - val_loss: 1.5907 - val_Accuracy: 0.6458 - lr: 0.0100
Epoch 4/10
422/422 [==============================] - 65s 153ms/step - loss: 1.7368 - Accuracy: 0.5289 - val_loss: 1.9703 - val_Accuracy: 0.6667 - lr: 0.0100
Epoch 5/10
422/422 [==============================] - 64s 152ms/step - loss: 1.6582 - Accuracy: 0.5459 - val_loss: 1.2954 - val_Accuracy: 0.7292 - lr: 0.0100
Epoch 6/10
422/422 [==============================] - 64s 151ms/step - loss: 1.6156 - Accuracy: 0.5557 - val_loss: 1.4720 - val_Accuracy: 0.7500 - lr: 0.0100
Epoch 7/10
422/422 [==============================] 

In [ ]:
model.fit(train,validation_data=val,epochs=10,callbacks=[earlystop,lr], steps_per_epoch=len(train), validation_steps=int(len(val)*0.1))

Epoch 1/10
422/422 [==============================] - 72s 170ms/step - loss: 1.2125 - Accuracy: 0.6469 - val_loss: 0.8904 - val_Accuracy: 0.8750 - lr: 0.0020
Epoch 2/10
422/422 [==============================] - 67s 158ms/step - loss: 1.1812 - Accuracy: 0.6535 - val_loss: 0.9799 - val_Accuracy: 0.8542 - lr: 0.0020
Epoch 3/10
422/422 [==============================] - ETA: 0s - loss: 1.1509 - Accuracy: 0.6582
Epoch 3: ReduceLROnPlateau reducing learning rate to 0.0003999999724328518.
422/422 [==============================] - 63s 150ms/step - loss: 1.1509 - Accuracy: 0.6582 - val_loss: 0.9369 - val_Accuracy: 0.8333 - lr: 0.0020
Epoch 4/10
422/422 [==============================] - 66s 156ms/step - loss: 1.1127 - Accuracy: 0.6680 - val_loss: 0.9432 - val_Accuracy: 0.8125 - lr: 4.0000e-04
Epoch 5/10
422/422 [==============================] - ETA: 0s - loss: 1.1064 - Accuracy: 0.6728
Epoch 5: ReduceLROnPlateau reducing learning rate to 7.999999215826393e-05.
422/422 [======================

In [ ]:
train_transforms = v2.Compose([
    v2.Resize(224),
    v2.RandomHorizontalFlip(),
    v2.RandomRotation(degrees=(0,15)),
    v2.ToTensor(),
    v2.Normalize(
        [0.485, 0.456, 0.406],
        [0.229,0.224,0.225]
    )
])

train_df = datasets.ImageFolder(
    root = r'/content/drive/MyDrive/sports/train',
    transform= train_transforms,
    )


val_transforms = v2.Compose([
    v2.Resize(224),
    v2.ToTensor(),
    v2.Normalize(
        [0.485, 0.456, 0.406],
        [0.229,0.224,0.225]
    )
])

val_df = datasets.ImageFolder(
    root = r'/content/drive/MyDrive/sports/valid',
    transform= val_transforms,
    )

/usr/local/lib/python3.10/dist-packages/torchvision/transforms/v2/_deprecated.py:43: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.
  warnings.warn(


In [ ]:
train_loader = torch.utils.data.DataLoader(train_df, batch_size=32, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_df, batch_size=100, shuffle=False)

In [ ]:
class CNNet(nn.Module):
  def __init__(self):
    super(CNNet, self).__init__()
    self.conv1 = nn.Sequential(
      nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, padding='same'),
      nn.ReLU(),
      nn.BatchNorm2d(64),
      nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding='same'),
      nn.ReLU(),
      nn.BatchNorm2d(64),
      nn.MaxPool2d(2),
      nn.Dropout(p=0.2)
    ) #224 -> 112
    self.conv2 = nn.Sequential(
      nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding='same'),
      nn.ReLU(),
      nn.BatchNorm2d(128),
      nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding='same'),
      nn.ReLU(),
      nn.BatchNorm2d(128),
      nn.MaxPool2d(2),
      nn.Dropout(p=0.2)
    ) #112 -> 56
    self.conv3 = nn.Sequential(
      nn.Conv2d(in_channels=128, out_channels=64, kernel_size=3, padding='same'),
      nn.ReLU(),
      nn.BatchNorm2d(64),
      nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding='same'),
      nn.ReLU(),
      nn.BatchNorm2d(64),
      nn.MaxPool2d(2),
      nn.Dropout(p=0.2)
    ) #56 -> 28
    self.func1 = nn.Sequential(
        nn.Flatten(),
        nn.Linear(64*28*28, 512),
        nn.ReLU(),
        nn.BatchNorm1d(512),
        nn.Dropout(p=0.2)
    )
    self.func2 = nn.Sequential(
        nn.Linear(512, 256),
        nn.ReLU(),
        nn.BatchNorm1d(256),
        nn.Dropout(p=0.2)
    )
    self.func3 = nn.Linear(256, 100)

  def forward(self, x):
    x = self.conv1(x)
    x = self.conv2(x)
    x = self.conv3(x)
    x = self.func1(x)
    x = self.func2(x)
    x = self.func3(x)
    out = nn.functional.log_softmax(x, dim=1)
    return out

In [ ]:
device = torch.device('cuda')
model = CNNet().to(device)
loss_function = nn.NLLLoss() #for logsoftmax
optimizer = optim.Adam(model.parameters(), lr = 0.001)

def train(
    model, device=None, train_loader=train_loader, val_loader=val_loader,
    loss_function=loss_function, optimizer=optimizer, epochs=10
  ):
  train_loss = []
  train_accuracy = []
  val_loss = []
  val_accuracy = []

  for e in range(epochs):
    model.train()
    total = 0
    losses = 0.0
    corrects = 0
    train_running_loss = 0
    for i, (X,y) in enumerate(train_loader):
      if device:
        X,y = X.to(device), y.to(device)
      optimizer.zero_grad()
      output = model(X)
      loss = loss_function(output, y)
      loss.backward()
      optimizer.step()
      _, pred = torch.max(output.data, 1)

      losses += loss.item()/X.size(0)
      corrects += torch.sum(pred == y.data)/X.size(0)
    train_loss.append(losses/len(train_loader))
    train_accuracy.append(corrects/len(train_loader))

    if True:
      model.eval()
      lossest = 0.0
      correctst = 0
      for Xt, yt in val_loader:
        with torch.no_grad():
          if device:
            Xt,yt = Xt.to(device), yt.to(device)
          outputt = model(Xt)
          losst = loss_function(outputt, yt)
          _, predt = torch.max(outputt.data, 1)
          lossest += losst.item()/Xt.size(0)
          correctst += torch.sum(predt == yt.data)/Xt.size(0)
      val_loss.append(lossest/len(val_loader))
      val_accuracy.append(correctst.double()/len(val_loader))
      print(f"epochs: {e}, Train loss: {train_loss[e]:.4f}, Val loss: {val_loss[e]:.4f}")
  return train_accuracy,val_accuracy

In [ ]:
acct

[tensor(0.2740, device='cuda:0', dtype=torch.float64),
 tensor(0.3860, device='cuda:0', dtype=torch.float64),
 tensor(0.4800, device='cuda:0', dtype=torch.float64),
 tensor(0.5160, device='cuda:0', dtype=torch.float64),
 tensor(0.5800, device='cuda:0', dtype=torch.float64),
 tensor(0.5340, device='cuda:0', dtype=torch.float64),
 tensor(0.5760, device='cuda:0', dtype=torch.float64),
 tensor(0.6180, device='cuda:0', dtype=torch.float64),
 tensor(0.6300, device='cuda:0', dtype=torch.float64),
 tensor(0.6220, device='cuda:0', dtype=torch.float64)]

In [ ]:
acc, acct = train(model, device=device, epochs=10)

epochs: 0, Train loss: 0.0236, Val loss: 0.0142
epochs: 1, Train loss: 0.0174, Val loss: 0.0146
epochs: 2, Train loss: 0.0156, Val loss: 0.0149
epochs: 3, Train loss: 0.0140, Val loss: 0.0154
epochs: 4, Train loss: 0.0124, Val loss: 0.0157
epochs: 5, Train loss: 0.0117, Val loss: 0.0146
epochs: 6, Train loss: 0.0106, Val loss: 0.0153
epochs: 7, Train loss: 0.0115, Val loss: 0.0151
epochs: 8, Train loss: 0.0094, Val loss: 0.0151
epochs: 9, Train loss: 0.0087, Val loss: 0.0159
